In [1]:
import numpy as numpy
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow.keras as tf_keras

In [7]:
df = pd.read_csv(r"C:\Work\2024\minion\Workspace\ml_basic\data\SMSSpamCollection.tsv", sep="\t", header=None, names= ["category", "sequence"])

In [8]:
df

,category,sequence
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [20]:
df["label"] = df["category"].replace({"ham": "0", "spam": "1"})

In [21]:
df.head()

,category,sequence,new_category,label
0,ham,"Go until jurong point, crazy.. Available only ...",0,0
1,ham,Ok lar... Joking wif u oni...,0,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1,1
3,ham,U dun say so early hor... U c already then say...,0,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0,0


In [22]:
df2 = df[["sequence", "label"]]

In [24]:
len(df2["sequence"]) * 0.8

4457.6

In [55]:
texts = df["sequence"].values
labels = df["label"].astype(int).values

In [56]:
raw_dataset = tf.data.Dataset.from_tensor_slices((texts, labels))
raw_dataset = raw_dataset.shuffle(buffer_size=3).batch(32)

In [57]:
max_tokens = 100000
sequence_length = 300

vectorize_layer = tf_keras.layers.TextVectorization(max_tokens= max_tokens,
                                                    output_mode = 'int',
                                                    output_sequence_length = sequence_length)

In [58]:
vectorize_layer.adapt(texts)

In [59]:
def vectorize_text(text, label):
    # text = tf.expand_dims(text, -1)    
    return vectorize_layer(text), label

In [60]:
vectorize_dataset = raw_dataset.map(vectorize_text)

In [61]:
for x, y in vectorize_dataset:
    print(x.shape)
    print(y[0])
    break

(32, 300)
tf.Tensor(0, shape=(), dtype=int32)


In [62]:
#모델구성
input = tf_keras.layers.Input(shape=(None,)) # 300 줘도 된다
x = tf_keras.layers.Embedding(input_dim = 100000, output_dim = 100)(input) # (300, 100)
x = tf_keras.layers.LSTM(units=16)(x)
output = tf_keras.layers.Dense(units = 1, activation="sigmoid")(x)

model = tf_keras.models.Model(inputs = input, outputs = output)

model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding_3 (Embedding)     (None, None, 100)         10000000  
                                                                 
 lstm_3 (LSTM)               (None, 16)                7488      
                                                                 
 dense_3 (Dense)             (None, 1)                 17        
                                                                 
Total params: 10007505 (38.18 MB)
Trainable params: 10007505 (38.18 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [63]:
model.compile(loss = 'binary_crossentropy', optimizer='adam', metrics= 'accuracy')

In [64]:
hisotry = model.fit(vectorize_dataset, epochs= 10)

Epoch 1/10
175/175 [==============================] - 27s 145ms/step - loss: 0.4121 - accuracy: 0.8659
Epoch 2/10
175/175 [==============================] - 26s 151ms/step - loss: 0.3949 - accuracy: 0.8659
Epoch 3/10
175/175 [==============================] - 25s 143ms/step - loss: 0.3948 - accuracy: 0.8659
Epoch 4/10
175/175 [==============================] - 25s 143ms/step - loss: 0.3949 - accuracy: 0.8659
Epoch 5/10
175/175 [==============================] - 25s 144ms/step - loss: 0.3946 - accuracy: 0.8659
Epoch 6/10
175/175 [==============================] - 25s 143ms/step - loss: 0.3946 - accuracy: 0.8659
Epoch 7/10
175/175 [==============================] - 25s 145ms/step - loss: 0.3947 - accuracy: 0.8659
Epoch 8/10
175/175 [==============================] - 24s 136ms/step - loss: 0.3945 - accuracy: 0.8659
Epoch 9/10
175/175 [==============================] - 24s 137ms/step - loss: 0.3945 - accuracy: 0.8659
Epoch 10/10
175/175 [==============================] - 26s 146ms/step - l